In [16]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import os
import cv2
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
from sklearn.preprocessing import MinMaxScaler


In [17]:
scaler = MinMaxScaler()

In [32]:
# Path to the directory containing the image tiles
input_folder = '/home/aayush/Documents/Aerial-UAV-Imagery-Segmentation/LandCoverSegmentation/Data/tiles'

# Path to the saved model
model_path = '/home/aayush/Documents/Aerial-UAV-Imagery-Segmentation/LandCoverSegmentation/models/landcover_99epochs.hdf5'
model = tf.keras.models.load_model(model_path,compile=False)


In [33]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [34]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((256, 256)) 
    img_np = np.array(img) / 255.0
    img_np = scaler.fit_transform(img_np.reshape(-1, img_np.shape[-1])).reshape(img_np.shape)
    img_np = preprocess_input(img_np)
    # print(img_np.shape)
    img_np = img_np.reshape((1, 256, 256, 3))

    return img_np

In [43]:
# Function to perform prediction on an image and save the prediction
def predict_and_save(image_path, output_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    # Perform prediction
    prediction = model.predict(img)  # Predict using the pre-trained model
    # prediction = np.argmax(prediction,axis=3)
    # prediction_img = Image.fromarray(prediction)
    prediction = np.squeeze(prediction)
    prediction_img = Image.fromarray((prediction[0] * 255).astype(np.uint8))  # Convert prediction to image format
    prediction_img.save(output_path)

In [44]:
for root, dirs, files in os.walk(input_folder):
    for filename in files:
        if filename.endswith('.png'):  
            tile_path = os.path.join(root, filename)
            # print(filename)
            # Generate output path for prediction (save the prediction in the same folder structure)
            output_folder = 'Pred/' 
            output_path = tile_path.replace(input_folder, output_folder)
            
            # Ensure the output directory exists, create if it doesn't
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # # Perform prediction and save the result
            predict_and_save(tile_path, output_path)


1/1 [==============================] - 0s 175ms/step
